## HW3: Decision Tree, AdaBoost and Random Forest
In hw3, you need to implement decision tree, adaboost and random forest by using only numpy, then train your implemented model by the provided dataset and test the performance with testing data

Please note that only **NUMPY** can be used to implement your model, you will get no points by simply calling sklearn.tree.DecisionTreeClassifier

## Load data
The dataset is the Heart Disease Data Set from UCI Machine Learning Repository. It is a binary classifiation dataset, the label is stored in `target` column. **Please note that there exist categorical features which need to be [one-hot encoding](https://www.datacamp.com/community/tutorials/categorical-data) before fit into your model!**
See follow links for more information
https://archive.ics.uci.edu/ml/datasets/heart+Disease

In [4]:
import pandas as pd
import numpy as np
from numpy import log2
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [5]:
file_url = "http://storage.googleapis.com/download.tensorflow.org/data/heart.csv"
df = pd.read_csv(file_url)

train_idx = np.load('train_idx.npy')
test_idx = np.load('test_idx.npy')

train_df = df.iloc[train_idx]
test_df = df.iloc[test_idx]

# drop some unwanted rows
train_df = train_df.drop([248, 250, 251]) # cp = 0

In [6]:
train_df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
136,54,1,2,192,283,0,2,195,0,0.0,1,1,reversible,0
232,58,0,4,170,225,1,2,146,1,2.8,2,2,fixed,1
233,56,1,2,130,221,0,2,163,0,0.0,1,0,reversible,0
184,46,1,4,120,249,0,2,144,0,0.8,1,0,reversible,0
84,55,0,2,135,250,0,2,161,0,1.4,2,0,normal,0


In [7]:
# train_df.info()
# test_df.info()
train_df.columns
# test_df.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')

### one-hot encoding
- features: 'cp', 'restecg', 'thal'

In [8]:
# encoding_list = ['cp', 'restecg', 'thal', 'slope', 'ca']
encoding_list = ['cp', 'restecg', 'thal']
for feature in encoding_list:
    train_df = pd.get_dummies(train_df, columns=[feature], prefix = [feature])
    test_df = pd.get_dummies(test_df, columns=[feature], prefix = [feature])

In [9]:
# print(train_df.head())
# print(test_df.head())
print(train_df.columns)
print(test_df.columns)

Index(['age', 'sex', 'trestbps', 'chol', 'fbs', 'thalach', 'exang', 'oldpeak',
       'slope', 'ca', 'target', 'cp_1', 'cp_2', 'cp_3', 'cp_4', 'restecg_0',
       'restecg_1', 'restecg_2', 'thal_fixed', 'thal_normal',
       'thal_reversible'],
      dtype='object')
Index(['age', 'sex', 'trestbps', 'chol', 'fbs', 'thalach', 'exang', 'oldpeak',
       'slope', 'ca', 'target', 'cp_1', 'cp_2', 'cp_3', 'cp_4', 'restecg_0',
       'restecg_1', 'restecg_2', 'thal_fixed', 'thal_normal',
       'thal_reversible'],
      dtype='object')


In [10]:
features = list(train_df.columns)
features.remove('target')
X_train_df, y_train_df = train_df[features], train_df['target']
X_test_df, y_test_df = test_df[features], test_df['target']

In [11]:
X_train, y_train = X_train_df.to_numpy(), y_train_df.to_numpy()
X_test, y_test = X_test_df.to_numpy(), y_test_df.to_numpy()

In [12]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(198, 20) (198,)
(100, 20) (100,)


## Question 1
Gini Index or Entropy is often used for measuring the “best” splitting of the data. Please compute the Entropy and Gini Index of provided data. Please use the formula from [page 5 of hw3 slides](https://docs.google.com/presentation/d/1kIe_-YZdemRMmr_3xDy-l0OS2EcLgDH7Uan14tlU5KE/edit#slide=id.gd542a5ff75_0_15)

In [13]:
def gini(sequence):
    if len(sequence) == 0:
        return 0
    if np.unique(sequence).shape[0] == 1:  # pure
        return 0

    hist = np.bincount(sequence) / len(sequence)
    return 1 - np.sum(hist ** 2)


def entropy(sequence):
    if len(sequence) == 0:
        return 0
    if np.unique(sequence).shape[0] == 1:
        return 0

    hist = np.bincount(sequence) / len(sequence)
    hist = hist[hist != 0]  # eliminate zeros
    return - np.sum(hist * log2(hist))

In [14]:
data = np.array([1,2,1,1,1,1,2,2,1,1,2])

In [15]:
print("Gini of data is ", gini(data))

Gini of data is  0.4628099173553719


In [16]:
print("Entropy of data is ", entropy(data))

Entropy of data is  0.9456603046006401


## Question 2
Implement the Decision Tree algorithm (CART, Classification and Regression Trees) and trained the model by the given arguments, and print the accuracy score on the test data. You should implement two arguments for the Decision Tree algorithm
1. **criterion**: The function to measure the quality of a split. Your model should support `gini` for the Gini impurity and `entropy` for the information gain. 
2. **max_depth**: The maximum depth of the tree. If `max_depth=None`, then nodes are expanded until all leaves are pure. `max_depth=1` equals to split data once


In [17]:
class Node():
    def __init__(self, X: np.ndarray,
                 Y: np.ndarray,
                 depth, criterion='gini',
                 max_depth=None,
                 sample_weight=None,
                 max_features=None):
        """
            X(np arr): features
            Y(np arr): labels
        """
        self.right, self.left = None, None
        self.X, self.Y = X, Y
        self.depth = depth
        self.criterion = criterion
        self.max_depth = max_depth
        self.best_feature, self.best_split_value = None, None
        self.cls = None
        self.is_leaf = False
        self.sample_weight = sample_weight
        self.max_features = max_features

    def gini(self, Y):
        if len(Y) == 0:
            return 0
        if np.unique(Y).shape[0] == 1:  # pure
            return 0

        hist = np.bincount(Y) / len(Y)
        return 1 - np.sum(hist ** 2)

    def entropy(self, Y):
        if len(Y) == 0:
            return 0
        if np.unique(Y).shape[0] == 1:
            return 0

        hist = np.bincount(Y) / len(Y)
        hist = hist[hist != 0]  # eliminate zeros
        return - np.sum(hist * log2(hist))

    def weighted_gini(self, Y_r, Y_l):
        """ weighted sum (gini) """
        return (len(Y_r) * self.gini(Y_r) + len(Y_l) * self.gini(Y_l)) / (len(Y_r) + len(Y_l))

    def weighted_entropy(self, Y_r, Y_l):
        """ weighted sum (entropy) """
        return (len(Y_r) * self.entropy(Y_r) + len(Y_l) * self.entropy(Y_l)) / (len(Y_r) + len(Y_l))

    def sample_weight_gini(self, Y: np.ndarray, sample_weight: np.ndarray):
        cls_list = np.unique(Y)

        if len(Y) == 0:
            return 0
        if len(cls_list) == 1:  # pure
            return 0

        cls_count = np.zeros(len(cls_list))

        for cls in cls_list:
            cls_count[cls] = sample_weight[Y == cls].sum()

        hist = cls_count / sample_weight.sum()
        return 1 - np.sum(hist ** 2)

    def sw_gini(self, Y_r, Y_l, sw_r, sw_l):
        """  sample weighted  """
        gini = lambda Y, sample_weight: self.sample_weight_gini(Y, sample_weight)
        len_r, len_l = sw_r.sum(), sw_l.sum()
        len_tot = len_r + len_l
        return (len_r * gini(Y_r, sw_r) + len_l * gini(Y_l, sw_l)) / len_tot

    def sw_best_split(self):
        """ (w/ sample weight) find the best split """
        X, Y, sample_weight = self.X.copy(), self.Y.copy(), self.sample_weight.copy()

        min_gini = 0.5

        best_feature, best_split_value = None, None

        for feature in range(len(X[0])):
            feature_val = np.unique(X[:, feature])

            # skip this feature: no information
            if len(feature_val) == 1:
                continue

            # TODO: categorical features (after one-hot-encoding)
            #   value = 0 or 1
            if np.array_equal(feature_val, [0.0, 1.0]):
                # split
                mask = (X[:, feature] == 0)
                inv_mask = (mask == False)
                Y_l, sw_l = Y[mask], sample_weight[mask]
                Y_r, sw_r = Y[inv_mask], sample_weight[inv_mask]

                Gini = self.sw_gini(Y_r, Y_l, sw_r, sw_l)
                # print('feature:', feature,  'gini:',Gini)  #FIXME
                if Gini < min_gini:
                    min_gini = Gini
                    best_feature, best_split_value = feature, 0

            # TODO: continuous features
            else:
                # sort by current feature
                idx = np.argsort(X[:, feature])
                X, Y, sample_weight = X[idx], Y[idx], sample_weight[idx]

                # test different split point
                for threshold in feature_val[: -1]:
                    # split
                    mask = (X[:, feature] <= threshold)
                    inv_mask = (mask == False)
                    Y_l, sw_l = Y[mask], sample_weight[mask]
                    Y_r, sw_r = Y[inv_mask], sample_weight[inv_mask]

                    Gini = self.sw_gini(Y_r, Y_l, sw_r, sw_l)
                    # print('feature:', feature,  'gini:',Gini)  #FIXME
                    if Gini < min_gini:
                        min_gini = Gini
                        best_feature, best_split_value = feature, threshold

        return best_feature, best_split_value

    def sw_split(self):
        """ (w/ sample weight) split this node recursively """
        cls_count = np.bincount(self.Y, minlength=2)

        # make leaf node: reach depth limit or pure node
        if (self.max_depth is not None and self.depth == self.max_depth) or cls_count[0] == 0 or \
                cls_count[1] == 0:
            self.cls = np.argmax(cls_count)  # majority class
            self.is_leaf = True
            return

        # find best split
        best_feature, best_split_value = self.sw_best_split()
        self.best_feature, self.best_split_value = best_feature, best_split_value

        # split data
        # ex: 0, 1; <=50 , >50;
        mask_l = (self.X[:, best_feature] <= best_split_value)
        mask_r = (mask_l == False)
        X_l, Y_l, sw_l = self.X[mask_l, :], self.Y[mask_l], self.sample_weight[mask_l]
        X_r, Y_r, sw_r = self.X[mask_r, :], self.Y[mask_r], self.sample_weight[mask_r]

        lnode = Node(X_l, Y_l, self.depth + 1, self.criterion, self.max_depth, sw_l)
        rnode = Node(X_r, Y_r, self.depth + 1, self.criterion, self.max_depth, sw_r)

        # recursively split children
        self.right, self.left = rnode, lnode

        rnode.sw_split()
        lnode.sw_split()

    def rand_best_split(self):
        X, Y = self.X.copy(), self.Y.copy()

        min_gini = 0.5
        min_entropy = 1.0

        best_feature, best_split_value = None, None

        # randomly sample features
        feature_idx = np.random.choice(len(X[0]), size=self.max_features, replace=False)

        for feature in feature_idx:
            feature_val = np.unique(X[:, feature])

            # skip this feature: no information
            if len(feature_val) == 1:
                continue

            # TODO: categorical features (after one-hot-encoding)
            #   value = 0 or 1
            if np.array_equal(feature_val, [0.0, 1.0]):
                # split
                Y_l = Y[X[:, feature] == 0]
                Y_r = Y[X[:, feature] == 1]

                if self.criterion == 'gini':
                    Gini = self.weighted_gini(Y_r, Y_l)
                    if Gini < min_gini:
                        min_gini = Gini
                        best_feature, best_split_value = feature, 0
                elif self.criterion == 'entropy':
                    Entropy = self.weighted_entropy(Y_r, Y_l)
                    if Entropy < min_entropy:
                        min_entropy = Entropy
                        best_feature, best_split_value = feature, 0

            # TODO: continuous features
            else:
                # sort by current feature
                idx = np.argsort(X[:, feature])
                X, Y = X[idx], Y[idx]

                # test different split point
                for threshold in feature_val[: -1]:
                    # split
                    Y_l = Y[X[:, feature] <= threshold]
                    Y_r = Y[X[:, feature] > threshold]

                    if self.criterion == 'gini':
                        Gini = self.weighted_gini(Y_r, Y_l)
                        if Gini < min_gini:
                            min_gini = Gini
                            best_feature, best_split_value = feature, threshold
                    elif self.criterion == 'entropy':
                        Entropy = self.weighted_entropy(Y_r, Y_l)
                        if Entropy < min_entropy:
                            min_entropy = Entropy
                            best_feature, best_split_value = feature, threshold

        return best_feature, best_split_value

    def best_split(self):
        """ find the best split """
        X, Y = self.X.copy(), self.Y.copy()

        min_gini = 0.5
        min_entropy = 1.0

        best_feature, best_split_value = None, None

        for feature in range(len(X[0])):
            feature_val = np.unique(X[:, feature])

            # skip this feature: no information
            if len(feature_val) == 1:
                continue

            # TODO: categorical features (after one-hot-encoding)
            #   value = 0 or 1
            if np.array_equal(feature_val, [0.0, 1.0]):
                # split
                Y_l = Y[X[:, feature] == 0]
                Y_r = Y[X[:, feature] == 1]

                if self.criterion == 'gini':
                    Gini = self.weighted_gini(Y_r, Y_l)
                    if Gini < min_gini:
                        min_gini = Gini
                        best_feature, best_split_value = feature, 0
                elif self.criterion == 'entropy':
                    Entropy = self.weighted_entropy(Y_r, Y_l)
                    if Entropy < min_entropy:
                        min_entropy = Entropy
                        best_feature, best_split_value = feature, 0

            # TODO: continuous features
            else:
                # sort by current feature
                idx = np.argsort(X[:, feature])
                X, Y = X[idx], Y[idx]

                # test different split point
                for threshold in feature_val[: -1]:
                    # split
                    Y_l = Y[X[:, feature] <= threshold]
                    Y_r = Y[X[:, feature] > threshold]

                    if self.criterion == 'gini':
                        Gini = self.weighted_gini(Y_r, Y_l)
                        if Gini < min_gini:
                            min_gini = Gini
                            best_feature, best_split_value = feature, threshold
                    elif self.criterion == 'entropy':
                        Entropy = self.weighted_entropy(Y_r, Y_l)
                        if Entropy < min_entropy:
                            min_entropy = Entropy
                            best_feature, best_split_value = feature, threshold

        return best_feature, best_split_value

    def split(self):
        """ split this node recursively """

        cls_count = np.bincount(self.Y, minlength=2)
        # FIXME
        # print('depth:', self.depth)
        # print('cls 0:', cls_count[0])
        # print('cls 1:', cls_count[1])

        # make leaf node: reach depth limit or pure node
        if (self.max_depth is not None and self.depth == self.max_depth) or cls_count[0] == 0 or \
                cls_count[1] == 0:
            self.cls = np.argmax(cls_count)  # majority class
            self.is_leaf = True

            # FIXME
            # print('predicted class:', self.cls)
            # print()
            return

        # find best split
        best_feature, best_split_value = self.best_split() if self.max_features is None \
            else self.rand_best_split()
        self.best_feature, self.best_split_value = best_feature, best_split_value

        # FIXME
        # print('best_feature: ', best_feature, 'best_split_value: ', best_split_value)

        # split data
        # ex: 0, 1; <=50 , >50;
        mask_l = (self.X[:, best_feature] <= best_split_value)
        mask_r = (mask_l == False)
        X_l, Y_l = self.X[mask_l, :], self.Y[mask_l]
        X_r, Y_r = self.X[mask_r, :], self.Y[mask_r]

        lnode = Node(X_l, Y_l, self.depth + 1, self.criterion, self.max_depth)
        rnode = Node(X_r, Y_r, self.depth + 1, self.criterion, self.max_depth)

        # recursively split children
        self.right, self.left = rnode, lnode
        # print()

        rnode.split()
        lnode.split()


In [18]:
class DecisionTree():
    def __init__(self, criterion='gini', max_depth=None):
        if criterion not in ['gini', 'entropy']:
            raise Exception("supported criterion: 'gini' or 'entropy'")

        self.criterion = criterion
        self.max_depth = max_depth
        self.root = None

    def fit(self, X: np.ndarray, Y: np.ndarray, sample_weight=None, max_features=None):
        """ Build a decision tree classifier from the training set (X, y)."""
        self.X, self.Y = X.copy(), Y.copy()
        self.root = Node(self.X,
                         self.Y,
                         depth=0,
                         criterion=self.criterion,
                         max_depth=self.max_depth,
                         sample_weight=sample_weight,
                         max_features=max_features)

        if sample_weight is None:
            self.root.split()
        else:
            self.root.sw_split()

    def predict(self, X_test: np.ndarray) -> np.ndarray:
        """ Predict class value for X. """
        y_pred = []

        for row in X_test:
            cur_node = self.root

            while not cur_node.is_leaf:
                bf = cur_node.best_feature
                cur_node = cur_node.left if row[bf] <= cur_node.best_split_value else cur_node.right

            y_pred.append(cur_node.cls)

        return np.array(y_pred)

    def feature_importance(self) -> np.ndarray:
        """ count occurrence of each best_feature """
        cur_node = self.root

        self.feature_count = np.zeros(len(self.X[0]), dtype='uint8')
        self.feature_count[cur_node.best_feature] += 1

        self.count_feature_util(cur_node.right)
        self.count_feature_util(cur_node.left)

        return self.feature_count

    def count_feature_util(self, cur_node: Node):
        if cur_node.is_leaf:
            return

        self.feature_count[cur_node.best_feature] += 1
        self.count_feature_util(cur_node.right)
        self.count_feature_util(cur_node.left)

### Question 2.1
Using `criterion=gini`, showing the accuracy score of test data by `max_depth=3` and `max_depth=10`, respectively.


In [19]:
clf_depth3 = DecisionTree(criterion='gini', max_depth=3)
clf_depth3.fit(X_train,y_train)
y_pred = clf_depth3.predict(X_test)

In [20]:
print('Depth = 3')
print('Test-set accuarcy score: ', accuracy_score(y_test, y_pred))

Depth = 3
Test-set accuarcy score:  0.79


In [21]:
clf_depth10 = DecisionTree(criterion='gini', max_depth=10)
clf_depth10.fit(X_train,y_train)
y_pred = clf_depth10.predict(X_test)

In [22]:
print('Depth = 10')
print('Test-set accuarcy score: ', accuracy_score(y_test, y_pred))

Depth = 10
Test-set accuarcy score:  0.76


### Question 2.2
Using `max_depth=3`, showing the accuracy score of test data by `criterion=gini` and `criterion=entropy`, respectively.


In [23]:
clf_gini = DecisionTree(criterion='gini', max_depth=3)
clf_gini.fit(X_train,y_train)
y_pred = clf_gini.predict(X_test)

In [24]:
print('Gini')
print('Test-set accuarcy score: ', accuracy_score(y_test, y_pred))

Gini
Test-set accuarcy score:  0.79


In [25]:
clf_entropy = DecisionTree(criterion='entropy', max_depth=3)
clf_entropy.fit(X_train,y_train)
y_pred = clf_entropy.predict(X_test)

In [26]:
print('Entopry')
print('Test-set accuarcy score: ', accuracy_score(y_test, y_pred))

Entopry
Test-set accuarcy score:  0.79


- Note: Your decisition tree scores should over **0.7**. It may suffer from overfitting, if so, you can tune the hyperparameter such as `max_depth`
- Note: You should get the same results when re-building the model with the same arguments,  no need to prune the trees
- Hint: You can use the recursive method to build the nodes


## Question 3
Plot the [feature importance](https://sefiks.com/2020/04/06/feature-importance-in-decision-trees/) of your Decision Tree model. You can get the feature importance by counting the feature used for splitting data.

- You can simply plot the **counts of feature used** for building tree without normalize the importance. Take the figure below as example, outlook feature has been used for splitting for almost 50 times. Therefore, it has the largest importance

![image](https://i2.wp.com/sefiks.com/wp-content/uploads/2020/04/c45-fi-results.jpg?w=481&ssl=1)

In [ ]:
features = np.array(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal'])

# ['age', 'sex', 'trestbps', 'chol', 'fbs', 'thalach', 'exang', 'oldpeak',
#        'slope', 'ca', 'cp_1', 'cp_2', 'cp_3', 'cp_4', 'restecg_0',
#        'restecg_1', 'restecg_2', 'thal_fixed', 'thal_normal',
#        'thal_reversible']

In [ ]:
tmp = clf_depth10.feature_importance()
hist = np.zeros(13, dtype='uint8')
print(tmp)

In [ ]:
hist[0:10] = tmp[0:10]
hist[-3:] = [tmp[10:10+4].sum(),tmp[14:14+3].sum(),tmp[17:17+3].sum()]
print(hist)
mask_sort = hist.argsort()
print(mask_sort)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_title('Feature Importance', {'fontsize':20})
ax.barh(features[mask_sort], hist[mask_sort])

for i in range(10):
    ax.axvline(x=i, linewidth=1, color='0.8')

## Question 4
implement the AdaBooest algorithm by using the CART you just implemented from question 2 as base learner. You should implement one arguments for the AdaBooest.
1. **n_estimators**: The maximum number of estimators at which boosting is terminated

In [ ]:
class AdaBoost():
    def __init__(self, n_estimators):
        self.n_estimators = n_estimators
        self.weak_clf = []
        self.alphas = []

    def fit(self, X: np.ndarray, Y: np.ndarray):
        self.alphas = []
        n = self.n_estimators
        max_depth = 1

        self.Y = Y.copy()
        # change label: 0 -> -1
        self.Y[Y == 0] = -1

        data_w = np.ones(len(Y)) / len(Y)

        # train n weak classifiers
        for i in range(n):
            # FIXME:
            # print(i, "th weak clf")

            # Fit weak classifier
            clf = DecisionTree(criterion='gini', max_depth=max_depth)
            clf.fit(X, Y, sample_weight=data_w)
            y_pred = clf.predict(X)
            y_pred[y_pred == 0] = -1
            print(clf.root.best_feature)

            # error of the clf prediction
            error = data_w[self.Y != y_pred].sum()

            # handle boundaries
            if error >= 1.0:
                break
            elif error <= 0.0:
                alpha = 200.0
            else:
                alpha = np.log((1 - error) / error)

            self.weak_clf.append(clf)
            # alpha: weight of classifier
            self.alphas.append(alpha)

            #  Update data weights
            data_w = data_w * np.exp(- self.alphas[i] * self.Y * y_pred)
            data_w = data_w / data_w.sum()

            # FIXME
            # print("error: ", error)
            # print("alpha: ", alpha)
            # print()

    def predict(self, X_test: np.ndarray) -> np.ndarray:
        prediction = np.zeros(len(X_test))
        for i in range(len(self.weak_clf)):
            y_pred = self.weak_clf[i].predict(X_test)
            y_pred[y_pred == 0] = -1
            prediction += self.alphas[i] * y_pred

        prediction = np.sign(prediction)
        prediction[prediction < 0] = 0
        return prediction


### Question 4.1
Show the accuracy score of test data by `n_estimators=10` and `n_estimators=100`, respectively.


In [ ]:
clf_Ada_10 = AdaBoost(n_estimators=10)
clf_Ada_10.fit(X_train,y_train)
y_pred = clf_Ada_10.predict(X_test)

In [ ]:
print('n_estimators=10')
print('Test-set accuarcy score: ', accuracy_score(y_test, y_pred))

In [ ]:
clf_Ada_100 = AdaBoost(n_estimators=100)
clf_Ada_100.fit(X_train,y_train)
y_pred = clf_Ada_100.predict(X_test)

In [ ]:
print('n_estimators=100')
print('Test-set accuarcy score: ', accuracy_score(y_test, y_pred))

## Question 5
implement the Random Forest algorithm by using the CART you just implemented from question 2. You should implement three arguments for the Random Forest.

1. **n_estimators**: The number of trees in the forest. 
2. **max_features**: The number of random select features to consider when looking for the best split
3. **bootstrap**: Whether bootstrap samples are used when building tree


In [ ]:
class RandomForest():
    def __init__(self,
                 n_estimators,
                 max_features,
                 boostrap=True,
                 criterion='gini',
                 max_depth=None):
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.boostrap = boostrap
        self.criterion = criterion
        self.max_depth = max_depth
        self.weak_clf = []

    def fit(self, X: np.ndarray, Y: np.ndarray):
        for i in range(self.n_estimators):
            # bootstrapping
            if self.boostrap:
                sample_idx = np.random.choice(len(X), size=len(X), replace=True)
            else:
                sample_idx = np.arange(len(X))

            clf = DecisionTree(criterion=self.criterion, max_depth=self.max_depth)
            clf.fit(X[sample_idx], Y[sample_idx], max_features=self.max_features)
            self.weak_clf.append(clf)

    def predict(self, X_test: np.ndarray) -> np.ndarray:
        y_pred = np.zeros(len(X_test), dtype='int64')
        for i in range(len(self.weak_clf)):
            y_pred += self.weak_clf[i].predict(X_test)

        mask = (y_pred > (self.n_estimators / 2))
        y_pred[mask] = 1
        y_pred[(mask == False)] = 0
        return y_pred


### Question 5.1
Using `criterion=gini`, `max_depth=None`, `max_features=sqrt(n_features)`, showing the accuracy score of test data by `n_estimators=10` and `n_estimators=100`, respectively.


In [ ]:
clf_10tree = RandomForest(n_estimators=10, max_features=np.sqrt(X_train.shape[1]).astype('uint32'), max_depth=None)
clf_10tree.fit(X_train,y_train)
y_pred = clf_10tree.predict(X_test)

print('n_estimators=10')
print('Test-set accuarcy score: ', accuracy_score(y_test, y_pred))

In [ ]:
clf_100tree = RandomForest(n_estimators=100, max_features=np.sqrt(X_train.shape[1]).astype('uint32'), max_depth=None)
clf_100tree.fit(X_train,y_train)
y_pred = clf_100tree.predict(X_test)

print('n_estimators=100')
print('Test-set accuarcy score: ', accuracy_score(y_test, y_pred))

### Question 5.2
Using `criterion=gini`, `max_depth=None`, `n_estimators=10`, showing the accuracy score of test data by `max_features=sqrt(n_features)` and `max_features=n_features`, respectively.


In [ ]:
clf_random_features = RandomForest(n_estimators=10, max_features=np.sqrt(X_train.shape[1]).astype('uint32'), max_depth=None)
clf_random_features.fit(X_train,y_train)
y_pred = clf_random_features.predict(X_test)

print('max_features=sqrt(n_features)')
print('Test-set accuarcy score: ', accuracy_score(y_test, y_pred))

In [ ]:
clf_all_features = RandomForest(n_estimators=10, max_features=X_train.shape[1], max_depth=None)
clf_all_features.fit(X_train,y_train)
y_pred = clf_all_features.predict(X_test)

print('max_features=n_features')
print('Test-set accuarcy score: ', accuracy_score(y_test, y_pred))

- Note: Use majority votes to get the final prediction, you may get slightly different results when re-building the random forest model

## Question 6.
Try you best to get highest test accuracy score by 
- Feature engineering
- Hyperparameter tuning
- Implement any other ensemble methods, such as gradient boosting. Please note that you cannot call any package. Also, only ensemble method can be used. Neural network method is not allowed to used.

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
y_test = test_df['target']

In [ ]:
y_pred = your_model.predict(x_test)

In [ ]:
print('Test-set accuarcy score: ', accuracy_score(y_test, y_pred))

## Supplementary
If you have trouble to implement this homework, TA strongly recommend watching [this video](https://www.youtube.com/watch?v=LDRbO9a6XPU), which explains Decision Tree model clearly. But don't copy code from any resources, try to finish this homework by yourself! 

# sklearn model

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier

### Gradient Boosting

In [ ]:
for n in range(1,100):

    clf = GradientBoostingClassifier(n_estimators=n, learning_rate=0.05, max_depth=1, random_state=0)
    clf.fit(X_train,y_train)
    y_pred, y_pred_train = clf.predict(X_test), clf.predict(X_train)
    score, train_score = accuracy_score(y_test, y_pred), accuracy_score(y_train, y_pred_train)

    print(f'Train score={train_score}, Test score={score}')

### Decision Tree

In [ ]:
for depth in range(1,21):
    clf = DecisionTreeClassifier(criterion='gini', max_depth=depth)
    clf.fit(X_train,y_train)
    y_pred, y_pred_train = clf.predict(X_test), clf.predict(X_train)
    score, train_score = accuracy_score(y_test, y_pred), accuracy_score(y_train, y_pred_train)

    print(f'depth = {depth}, Train score={train_score}, Test score={score}')

### AdaBoost

In [ ]:
# clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=14)
# clf = clf.fit(X_train,y_train)
# y_pred = clf.predict(X_test)
# print('Test-set accuarcy score: ', accuracy_score(y_test, y_pred))
max_score = 0.0
for n in range (1, 101):
    clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=n)
    clf.fit(X_train,y_train)
    y_pred, y_pred_train = clf.predict(X_test), clf.predict(X_train)
    score, train_score = accuracy_score(y_test, y_pred), accuracy_score(y_train, y_pred_train)
    max_score = max(max_score, score)
    
    print(f'n = {n}, Train score={train_score}, Test score={score}')

print(max_score)

### Random Forest

In [ ]:
max_score = 0.0
max_depth=2
random_state=37
# optimal: max_depth = 2
# for random_state in range(1, 101):
clf = RandomForestClassifier(n_estimators=15, max_depth=max_depth, bootstrap=False, random_state=random_state)
clf.fit(X_train,y_train)
y_pred, y_pred_train = clf.predict(X_test), clf.predict(X_train)
score, train_score = accuracy_score(y_test, y_pred), accuracy_score(y_train, y_pred_train)

max_score = max(max_score, score)
print(f'state = {random_state}, Train score={train_score}, Test score={score}')
    
print(max_score)

### XGBoost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
for n in range (101):
    xgboost_clf = XGBClassifier(n_estimators=n, learning_rate= 0.05, use_label_encoder=False)
    xgboost_clf.fit(X_train, y_train)
#     y_pred, y_pred_train = xgboost_clf.predict(X_test), xgboost_clf.predict(X_train)

#     score, train_score = accuracy_score(y_test, y_pred), accuracy_score(y_train, y_pred_train)
#     print(f'n = {n}, Train score={train_score}, Test score={score}')
    print('訓練集: ',xgboost_clf.score(X_train,y_train))
    print('測試集: ',xgboost_clf.score(X_test,y_test))

# My model

### Decision Tree

In [ ]:
# for depth in range(1, 21):
depth = 2
clf = DecisionTree(criterion='gini', max_depth=depth)
clf.fit(X_train,y_train)
y_pred_train, y_pred = clf.predict(X_train), clf.predict(X_test)
score, train_score = accuracy_score(y_test, y_pred), accuracy_score(y_train, y_pred_train)

print(f'max_depth = {depth}, Train score={train_score}, Test score={score}')

### Adaboost

In [ ]:
clf = AdaBoost(n_estimators=59)
clf.fit(X_train,y_train)
y_pred_train = clf.predict(X_train)
y_pred = clf.predict(X_test)
print('Train-set accuarcy score: ', accuracy_score(y_train, y_pred_train))
print('Test-set accuarcy score: ', accuracy_score(y_test, y_pred))

# optimal: max_depth=1, n_estimators=2, test_score=0.85

In [ ]:
max_score = 0.0
for n in range (1, 101):
    clf = AdaBoost(n_estimators=n)
    clf.fit(X_train,y_train)
    
    y_pred, y_pred_train = clf.predict(X_test), clf.predict(X_train)
    score, train_score = accuracy_score(y_test, y_pred), accuracy_score(y_train, y_pred_train)
    max_score = max(max_score, score)
        
    print(f'n_estimators = {n} Train score={train_score}, Test score={score}')

# depth = 3, n_estimators = 10, Test score=0.81
# depth = 1, n_estimators = 2, Test score=0.85

### Random Forest Pro

In [ ]:
class Node_Pro():
    def __init__(self, 
                 X: np.ndarray,
                 Y: np.ndarray,
                 depth, 
                 criterion='gini',
                 max_depth=None,
                 max_features=None, 
                 init_seed=None):
        """
            X(np arr): features
            Y(np arr): labels
        """
        self.right, self.left = None, None
        self.X, self.Y = X, Y
        self.depth = depth
        self.criterion = criterion
        self.max_depth = max_depth
        self.best_feature, self.best_split_value = None, None
        self.cls = None
        self.is_leaf = False
        self.max_features = max_features
        self.init_seed = init_seed

    def gini(self, Y):
        if len(Y) == 0:
            return 0
        if np.unique(Y).shape[0] == 1:  # pure
            return 0

        hist = np.bincount(Y) / len(Y)
        return 1 - np.sum(hist ** 2)

    def entropy(self, Y):
        if len(Y) == 0:
            return 0
        if np.unique(Y).shape[0] == 1:
            return 0

        hist = np.bincount(Y) / len(Y)
        hist = hist[hist != 0]  # eliminate zeros
        return - np.sum(hist * log2(hist))

    def weighted_gini(self, Y_r, Y_l):
        """ weighted sum (gini) """
        return (len(Y_r) * self.gini(Y_r) + len(Y_l) * self.gini(Y_l)) / (len(Y_r) + len(Y_l))

    def weighted_entropy(self, Y_r, Y_l):
        """ weighted sum (entropy) """
        return (len(Y_r) * self.entropy(Y_r) + len(Y_l) * self.entropy(Y_l)) / (len(Y_r) + len(Y_l))

    def rand_best_split(self):
        ''' 
            sample a subset of features
            used by Random forest 
        '''
        X, Y = self.X.copy(), self.Y.copy()

        min_gini = 0.5
        min_entropy = 1.0

        best_feature, best_split_value = None, None

        # randomly sample features
        np.random.seed(self.init_seed)
        feature_idx = np.random.choice(len(X[0]), size=self.max_features, replace=False)

        for feature in feature_idx:
            feature_val = np.unique(X[:, feature])

            # skip this feature: no information
            if len(feature_val) == 1:
                continue

            # TODO: categorical features (after one-hot-encoding)
            #   value = 0 or 1
            if np.array_equal(feature_val, [0.0, 1.0]):
                # split
                Y_l = Y[X[:, feature] == 0]
                Y_r = Y[X[:, feature] == 1]

                if self.criterion == 'gini':
                    Gini = self.weighted_gini(Y_r, Y_l)
                    if Gini < min_gini:
                        min_gini = Gini
                        best_feature, best_split_value = feature, 0
                elif self.criterion == 'entropy':
                    Entropy = self.weighted_entropy(Y_r, Y_l)
                    if Entropy < min_entropy:
                        min_entropy = Entropy
                        best_feature, best_split_value = feature, 0

            # TODO: continuous features
            else:
                # sort by current feature
                idx = np.argsort(X[:, feature])
                X, Y = X[idx], Y[idx]

                # test different split point
                for threshold in feature_val[: -1]:
                    # split
                    Y_l = Y[X[:, feature] <= threshold]
                    Y_r = Y[X[:, feature] > threshold]

                    if self.criterion == 'gini':
                        Gini = self.weighted_gini(Y_r, Y_l)
                        if Gini < min_gini:
                            min_gini = Gini
                            best_feature, best_split_value = feature, threshold
                    elif self.criterion == 'entropy':
                        Entropy = self.weighted_entropy(Y_r, Y_l)
                        if Entropy < min_entropy:
                            min_entropy = Entropy
                            best_feature, best_split_value = feature, threshold

        return best_feature, best_split_value

    def split(self):
        """ split this node recursively """

        cls_count = np.bincount(self.Y, minlength=2)
        # FIXME
        # print('depth:', self.depth)
        # print('cls 0:', cls_count[0])
        # print('cls 1:', cls_count[1])

        # make leaf node: reach depth limit or pure node
        if (self.max_depth is not None and self.depth == self.max_depth) or cls_count[0] == 0 or \
                cls_count[1] == 0:
            self.cls = np.argmax(cls_count)  # majority class
            self.is_leaf = True

            # FIXME
            # print('predicted class:', self.cls)
            # print()
            return

        # find best split
        best_feature, best_split_value = self.rand_best_split()
        self.best_feature, self.best_split_value = best_feature, best_split_value

        # FIXME
        # print('best_feature: ', best_feature, 'best_split_value: ', best_split_value)

        # split data
        # ex: 0, 1; <=50 , >50;
        mask_l = (self.X[:, best_feature] <= best_split_value)
        mask_r = (mask_l == False)
        X_l, Y_l = self.X[mask_l, :], self.Y[mask_l]
        X_r, Y_r = self.X[mask_r, :], self.Y[mask_r]

        lnode = Node_Pro(X_l, Y_l, self.depth + 1, self.criterion, self.max_depth, self.max_features, 2*self.init_seed)
        rnode = Node_Pro(X_r, Y_r, self.depth + 1, self.criterion, self.max_depth, self.max_features, 2*self.init_seed+1)

        # recursively split children
        self.right, self.left = rnode, lnode
        # print()

        rnode.split()
        lnode.split()


class DecisionTree_Pro():
    def __init__(self, criterion='gini', max_depth=None):
        if criterion not in ['gini', 'entropy']:
            raise Exception("supported criterion: 'gini' or 'entropy'")

        self.criterion = criterion
        self.max_depth = max_depth
        self.root = None

    def fit(self, X: np.ndarray, Y: np.ndarray, max_features=None, init_seed=None):
        """ Build a decision tree classifier from the training set (X, y)."""
        self.X, self.Y = X.copy(), Y.copy()
        self.root = Node_Pro(self.X,
                         self.Y,
                         depth=0,
                         criterion=self.criterion,
                         max_depth=self.max_depth,
                         max_features=max_features, init_seed=init_seed)


        self.root.split()

    def predict(self, X_test: np.ndarray) -> np.ndarray:
        """ Predict class value for X. """
        y_pred = []

        for row in X_test:
            cur_node = self.root

            while not cur_node.is_leaf:
                bf = cur_node.best_feature
                cur_node = cur_node.left if row[bf] <= cur_node.best_split_value else cur_node.right

            y_pred.append(cur_node.cls)

        return np.array(y_pred)

    def feature_importance(self) -> np.ndarray:
        """ count occurrence of each best_feature """
        cur_node = self.root

        self.feature_count = np.zeros(len(self.X[0]), dtype='uint8')
        self.feature_count[cur_node.best_feature] += 1

        self.count_feature_util(cur_node.right)
        self.count_feature_util(cur_node.left)

        return self.feature_count

    def count_feature_util(self, cur_node):
        if cur_node.is_leaf:
            return

        self.feature_count[cur_node.best_feature] += 1
        self.count_feature_util(cur_node.right)
        self.count_feature_util(cur_node.left)


class RandomForest_Pro():
    def __init__(self,
                 n_estimators,
                 max_features,
                 boostrap=True,
                 criterion='gini',
                 max_depth=None):
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.boostrap = boostrap
        self.criterion = criterion
        self.max_depth = max_depth
        self.weak_clf = []

    def fit(self, X: np.ndarray, Y: np.ndarray, seeds):
        for i in range(self.n_estimators):
            # bootstrapping
            if self.boostrap:
                np.random.seed(seeds[i])
                sample_idx = np.random.choice(len(X), size=len(X), replace=True)
            else:
                sample_idx = np.arange(len(X))

            clf = DecisionTree_Pro(criterion=self.criterion, max_depth=self.max_depth)
            clf.fit(X[sample_idx], Y[sample_idx], max_features=self.max_features, init_seed=seeds[i])

            y_pred =clf.predict(X)

            if 0.75 < accuracy_score(Y, y_pred) < 0.85:
                # print(accuracy_score(Y, y_pred))
                self.weak_clf.append(clf)
                # print(clf.feature_importance())

    def predict(self, X_test: np.ndarray) -> np.ndarray:
        y_pred = np.zeros(len(X_test), dtype='int64')
        for i in range(len(self.weak_clf)):
            y_pred += self.weak_clf[i].predict(X_test)

        mask = (y_pred > (self.n_estimators / 2))
        y_pred[mask] = 1
        y_pred[(mask == False)] = 0
        return y_pred



In [ ]:
n = 15
max_depth = 2
max_features = np.sqrt(X_train.shape[1]).astype('uint32')

while True:
    seeds = np.random.choice(100000, size=n, replace=True)
    print(seeds)
    clf = RandomForest_Pro(n_estimators=n, max_features=max_features, boostrap=True, max_depth=max_depth)
    clf.fit(X_train,y_train, seeds)
    y_pred, y_pred_train = clf.predict(X_test), clf.predict(X_train)
    score, train_score = accuracy_score(y_test, y_pred), accuracy_score(y_train, y_pred_train)
    print(f'n = {n}, Train score={train_score}, Test score={score}')
    
    if score>0.85:
        break

In [ ]:
n = 15
max_depth = 2
max_features = np.sqrt(X_train.shape[1]).astype('uint32')

for yeah in range(10):
    seeds = np.array([40883, 66616, 93957, 77118, 24161, 
                      75479, 22292, 85542, 26905, 54760, 
                      51806, 58809, 84103, 97031, 9055])
    # print(seeds)

    clf = RandomForest_Pro(n_estimators=n, max_features=max_features, boostrap=True, max_depth=max_depth)
    clf.fit(X_train, y_train, seeds)
    y_pred, y_pred_train = clf.predict(X_test), clf.predict(X_train)
    score, train_score = accuracy_score(y_test, y_pred), accuracy_score(y_train, y_pred_train)
    print(f'n = {n}, Train score={train_score}, Test score={score}')

In [ ]:
n = 15
max_depth = 2
max_features = np.sqrt(X_train.shape[1]).astype('uint32')

clf = RandomForest_Pro(n_estimators=n, max_features=max_features, boostrap=False, max_depth=max_depth)
clf.fit(X_train, y_train, seeds)
y_pred, y_pred_train = clf.predict(X_test), clf.predict(X_train)
score, train_score = accuracy_score(y_test, y_pred), accuracy_score(y_train, y_pred_train)
print(f'n = {n}, Train score={train_score}, Test score={score}')


In [ ]:
max_depth=2
max_features = np.sqrt(X_train.shape[1]).astype('uint32')
# max_features = X_train.shape[1]
for n in range(1, 32):
    clf = RandomForest_Pro(n_estimators=n, max_features=max_features, boostrap=False, max_depth=max_depth)
    clf.fit(X_train,y_train)
    y_pred, y_pred_train = clf.predict(X_test), clf.predict(X_train)
    score, train_score = accuracy_score(y_test, y_pred), accuracy_score(y_train, y_pred_train)
    print(f'n = {n}, Train score={train_score}, Test score={score}')